# Importieren

In [32]:
# "Strg + Enter" um die Benutzereingabe anzuzeigen
import pickle
import math
import matplotlib.pyplot as plt
import numpy as np
import os,glob
import pathlib
import pandas as pd
import copy
from ipynb.fs.full.kistler_funktionen import k_import, k_mittelwerte, k_filter, k_start_ende, k_detrend 
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import configparser
import dill

### Funktionen ##################################

def importieren(b):
    
    output.clear_output()
        
    # Kraftsignale
    if k_reimport.value:
        k_import(trennzeichen = k_trennzeichen.value, speicherort = k_speicherort.value)
        k_reimport.value = False
    with open('Kraftsignale.pkl','rb') as f:  # aus Kraftsignalimport
        Zeit, Fx, Fy, Fz = pickle.load(f)
    del f

    # Mitschrift
    filename = m_speicherort.value + '\\' + m_dateiname.value
    mitschrift = pd.read_excel(filename, sheet_name=(m_blattnummer.value-1), header=0)
    
    # Fehlermeldung wenn die Anzahl der Messdaten und die Excel Mitschrift nicht zusammen passen
    if not np.shape(Zeit)[1] == len(mitschrift):
        raise Exception('Anzahl der Kraftsignale (np.shape(Zeit)[1]) und der Mitschrift (len(mitschrift)) nicht gleich! \n')  

    # Alicona Ergebnisse
    if a_import.value:
        filename = a_speicherort.value + '\\' + a_dateiname.value
        alicona = pd.read_excel(filename, sheet_name=(a_blattnummer.value-1), header=0)
        alicona.set_index('probennummer', inplace=True)
        
    with output:
        print("Import Erfolgreich.")
        
    #return locals()

    with open('import.pkl', 'wb') as file:
        if a_import.value:
            dill.dump((Zeit, Fx, Fy, Fz, mitschrift, alicona), file)
        else:
            dill.dump((Zeit, Fx, Fy, Fz, mitschrift), file)

    #mittelw_def_1.max = Zeit[0,-1]
    #mittelw_def_1
    
def speichern_f(b):    
    config['Import']['k_ordner'] = str(k_speicherort.value)
    config['Import']['k_trennzeichen'] = str(k_trennzeichen.value)
    config['Import']['m_speicherort'] = str(m_speicherort.value)
    config['Import']['m_dateiname'] = str(m_dateiname.value)
    config['Import']['m_blattnummer'] = str(m_blattnummer.value)
    config['Import']['a_speicherort'] = str(a_speicherort.value)
    config['Import']['a_dateiname'] = str(a_dateiname.value)
    config['Import']['a_blattnummer'] = str(a_blattnummer.value)
    
    with open('kistler_auswertung_config.ini', 'w') as configfile:
        config.write(configfile)
        
### Widgets definieren ##################################

# Config lesen umd Widgets richtig zu initialisieren
config = configparser.ConfigParser()
config.read('kistler_auswertung_config.ini')

# sytel muss sein
style = {'description_width': '120px'}    

# Widgets initialisieren
k_speicherort = widgets.Text(
    value= config['Import']['k_ordner'],
    description='Kistler Ordnerpfad:',
    layout = widgets.Layout(width='95%'),
    style = style)
k_trennzeichen = widgets.Dropdown(
    options=['tab', ';', '.', ':', '-'],
    value=config['Import']['k_trennzeichen'],
    description='Trennzeichen:',
    layout = widgets.Layout(width='20%'),
    disabled=False,
    style = style)
k_reimport = widgets.Checkbox(
    value=False,
    description='Kraftsignale neu importieren',
    disabled=False,
    indent=False)
m_speicherort = widgets.Text(
    value= config['Import']['m_speicherort'],
    description='Mitschrift Ordner:',
    layout = widgets.Layout(width='95%'),
    style = style)
m_dateiname = widgets.Text(
    value= config['Import']['m_dateiname'],
    description='Dateiname:',
    layout = widgets.Layout(width='50%'),
    style = style)
m_blattnummer = widgets.Dropdown(
    options=[1, 2, 3, 4],
    value= int(config['Import']['m_blattnummer']),
    description='Blattnummer:',
    layout = widgets.Layout(width='20%'),
    disabled=False,
    style = style)
a_import = widgets.Checkbox(
    value=False,
    description='Oberlfächenkennwerte importieren',
    disabled=False,
    indent=False)
a_speicherort = widgets.Text(
    value= config['Import']['a_speicherort'],
    description= 'Ordnerpfad',
    layout = widgets.Layout(width='95%'),
    style = style)
a_dateiname = widgets.Text(
    value= config['Import']['a_dateiname'],
    description= 'Dateiname',
    layout = widgets.Layout(width='50%'),
    style = style)
a_blattnummer = widgets.Dropdown(
    options=[1, 2, 3, 4],
    value= int(config['Import']['a_blattnummer']),
    description='Blattnummer:',
    layout = widgets.Layout(width='20%'),
    disabled=False,
    style = style)
importieren_w = widgets.Button(
    description='Importieren',
    layout = widgets.Layout(width='25%'))

# Tab initialisieren
tab_0 = VBox(children=[k_reimport,
               k_speicherort,
               k_trennzeichen,
               m_speicherort,
               m_dateiname,        
               m_blattnummer,
               a_import,
               a_speicherort,
               a_dateiname,
               a_blattnummer])
tab = widgets.Tab(children=[tab_0])
tab.set_title(0, 'Importieren')

# Outputfenster
output = widgets.Output()

# Erste Ausgabe
with output:
        print("Es wurde noch nichts Importiert.")

# Funktion der Widgets beim Klick
importieren_w.on_click(speichern_f)    
importieren_w.on_click(importieren)

### Rest der GUI ##################################

# Darstellung der Tabs muss in der letzten Zeile stehen
VBox(children=[tab, importieren_w, output])

# Berechnen

In [33]:
# "Strg + Enter" um die Benutzereingabe anzuzeigen
import pickle
import math
import matplotlib.pyplot as plt
import numpy as np
import os,glob
import pathlib
import pandas as pd
import copy
from ipynb.fs.full.kistler_funktionen import k_import, k_mittelwerte, k_filter, k_start_ende, k_detrend, k_peaks, k_maxwerte
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import configparser
import dill


### Daten aus dem Import lesen #################################
try:
    with open('import.pkl', 'rb') as file:
        [Zeit, Fx, Fy, Fz, mitschrift, alicona] = dill.load(file)
except:
    with open('import.pkl', 'rb') as file:
        [Zeit, Fx, Fy, Fz, mitschrift] = dill.load(file)

### Funktionen definieren ######################################       
        
def berechnen():

    # Falls neu Importiert wurde daten nochmal öffnen
    try:
        with open('import.pkl', 'rb') as file:
            [Zeit, Fx, Fy, Fz, mitschrift, alicona] = dill.load(file)
    except:
        with open('import.pkl', 'rb') as file:
            [Zeit, Fx, Fy, Fz, mitschrift] = dill.load(file)
    
    # Output der graphischen Darstellung löschen
    output.clear_output()
   
    # Zeitspahnvolumen berechnen
    q = (mitschrift.ap * mitschrift.ae * mitschrift.vc * mitschrift.fu) / (mitschrift.durchmesser * math.pi )
    q = np.round(q, decimals=2)

    # Standweg runden
    mitschrift.standweg = np.round(mitschrift.standweg, decimals=2)
    
    ## Auswertung der Kraftsignale
    # Vorzeichen korrigieren
    if Vorzeichen_Fx_check.value: Fx = -Fx
    if Vorzeichen_Fy_check.value: Fy = -Fy
    if Vorzeichen_Fz_check.value: Fz = -Fz

    # Offset bei start der Messung korrigieren
    if Offset_Fx_check.value:
        offset = copy.copy(Fx[0,:])
        Fx[:,:] += -Fx[0,:]
    if Offset_Fy_check.value:
        offset = copy.copy(Fy[0,:])
        Fy[:,:] += -Fy[0,:]
    if Offset_Fz_check.value:
        offset = copy.copy(Fz[0,:])
        Fz[:,:] += -Fz[0,:]   
    
    # Mittelwerte berechnen - [Fx_mean, Fy_mean, Fz_mean]    
    start = []
    ende = []
    
    # start und ende aus den Schiebereglern lesen
    for i in range(np.shape(Zeit)[1]):
        start.append(mittelwerte_def[i].value[0])
        ende.append(mittelwerte_def[i].value[1])
        
    with open('mittelwerte_def.pkl', 'wb') as file:
        dill.dump((start, ende), file)
    
    # Peaks
    [Fx_peaks, Fy_peaks, Fz_peaks] = k_peaks(Zeit, [Fx, Fy, Fz], start=start, ende=ende, distance = peak_abstand_w.value)
    [Fx_peaks_mean, Fy_peaks_mean, Fz_peaks_mean] = k_mittelwerte(Zeit, [Fx_peaks, Fy_peaks, Fz_peaks], start=start, ende=ende)
 
    # Mittelwerte
    [Fx_mean, Fy_mean, Fz_mean] = k_mittelwerte(Zeit, [Fx, Fy, Fz], start=start, ende=ende)
    
    # Maximallwerte
    [Fx_max, Fy_max, Fz_max] = k_maxwerte(Zeit, [Fx, Fy, Fz], start=start, ende=ende)

    # Filtern - [Fx_f, Fy_f]
    [Fx_f, Fy_f, Fz_f] = k_filter(Zeit, [Fx, Fy, Fz], filterwert=Filterwert.value)

    ## Ergebnisse zusammenfassen und speichern
    ergebnisse = mitschrift[['probe', 'werkzeug', 'korngroesse', 'standweg', 
                             'parameter', 'drehrichtung', 'fu', 'vc', 'ae', 'ap', 'probennummer']]
    ergebnisse = ergebnisse.assign(q = q)
    try:
        ergebnisse = ergebnisse.assign(start = start)
        ergebnisse = ergebnisse.assign(ende = ende)
    except:
        pass

    ergebnisse.set_index('probennummer', inplace=True)

    
    ergebnisse = ergebnisse.assign(Fx_mean = Fx_mean)
    ergebnisse = ergebnisse.assign(Fy_mean = Fy_mean)
    ergebnisse = ergebnisse.assign(Fz_mean = Fz_mean)
    ergebnisse = ergebnisse.assign(Fx_max = Fx_max)
    ergebnisse = ergebnisse.assign(Fy_max = Fy_max)
    ergebnisse = ergebnisse.assign(Fz_max = Fz_max)
    ergebnisse = ergebnisse.assign(Fx_peaks_mean = Fx_peaks_mean)
    ergebnisse = ergebnisse.assign(Fy_peaks_mean = Fy_peaks_mean)
    ergebnisse = ergebnisse.assign(Fz_peaks_mean = Fz_peaks_mean)    
    
    if "alicona" in locals():
        ergebnisse = pd.concat([ergebnisse, alicona], axis=1)
        del alicona

    filename = e_ordnerpfad.value + '\\' + e_dateiname.value
    ergebnisse.to_excel(filename, sheet_name='Python Ergebnisse')

    del mitschrift, q

    # Ausgabe für Graphische Oberfläche
    with output:
        display(ergebnisse.head(5))
        
    # Speichern
    with open('ergebnisse.pkl', 'wb') as file:
            dill.dump((Zeit, Fx, Fy, Fz, Fx_f, Fy_f, Fz_f, Fx_peaks, Fy_peaks, Fz_peaks, Fx_mean, Fy_mean, Fz_mean,
                       ergebnisse), file)        
    return locals()

def speichern_f():
    config['Import']['Vorzeichen_Fx_check'] = str(Vorzeichen_Fx_check.value)
    config['Import']['Vorzeichen_Fy_check'] = str(Vorzeichen_Fy_check.value)
    config['Import']['Vorzeichen_Fz_check'] = str(Vorzeichen_Fz_check.value)
    config['Import']['Offset_Fx_check'] = str(Offset_Fx_check.value)
    config['Import']['Offset_Fy_check'] = str(Offset_Fy_check.value)
    config['Import']['Offset_Fz_check'] = str(Offset_Fz_check.value)
    config['Import']['e_ordnerpfad'] = str(e_ordnerpfad.value)
    config['Import']['e_dateiname'] = str(e_dateiname.value)
    config['Import']['gesamt_mittel_min'] = str(mittelwerte_def_ges.value[0])
    config['Import']['gesamt_mittel_max'] = str(mittelwerte_def_ges.value[1])
    config['Import']['Peakabstand'] = str(Peakabstand.value)
    
    
    with open('kistler_auswertung_config.ini', 'w') as configfile:
        config.write(configfile)
        
    start = []
    ende = []
    
    for i in range(np.shape(Zeit)[1]):
        start.append(mittelwerte_def[i].value[0])
        ende.append(mittelwerte_def[i].value[1])
    
    with open('mittelwerte_def.pkl', 'wb') as file:
        dill.dump((start, ende), file)

def default_f():
    # default config lesen
    default = configparser.ConfigParser()
    default.read('kistler_auswertung_config_default.ini')
    
    # gelesene werte speichern
    with open('kistler_auswertung_config.ini', 'w') as configfile:
        default.write(configfile)
        
def mittelwerte_ges_f():
    
    for i in range(np.shape(Zeit)[1]):
        mittelwerte_def[i].value = mittelwerte_def_ges.value

## Initialisierung ######################################

# Config lesen
config = configparser.ConfigParser()
config.read('kistler_auswertung_config.ini')

# Style muss sein
style = {'description_width': '120px'}

# Korrekturen
Vorzeichen_Fx_check = widgets.Checkbox(
    value= config['Import']['Vorzeichen_Fx_check'] == 'True',
    description='Vorzeichenumkehr Fx',
    disabled=False,
    indent=False)
Vorzeichen_Fy_check = widgets.Checkbox(
    value= config['Import']['Vorzeichen_Fy_check'] == 'True',
    description='Vorzeichenumkehr Fy',
    disabled=False,
    indent=False)
Vorzeichen_Fz_check = widgets.Checkbox(
    value= config['Import']['Vorzeichen_Fz_check'] == 'True',
    description='Vorzeichenumkehr Fz',
    disabled=False,
    indent=False)
Offset_Fx_check = widgets.Checkbox(
    value= config['Import']['Offset_Fx_check'] == 'True',
    description='Offsetkorrektur Fx',
    disabled=False,
    indent=False)
Offset_Fy_check = widgets.Checkbox(
    value= config['Import']['Offset_Fy_check'] == 'True',
    description='Offsetkorrektur Fy',
    disabled=False,
    indent=False)
Offset_Fz_check = widgets.Checkbox(
    value= config['Import']['Offset_Fz_check'] == 'True',
    description='Offsetkorrektur Fz',
    disabled=False,
    indent=False)

# Zeitfenster Einzeln
anz = np.shape(Zeit)[1]
mittelwerte_def = list(range(anz))
for i in range(anz):
    mittelwerte_def[i] = widgets.FloatRangeSlider(
    value=[0, 1],
    min=0,
    max=Zeit[-1,0],
    step=0.1,
    description='Messung: ' + str(i+1),
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style = {'description_width': '100px'}
    )
    
# Mittelwerte Schieberegler lesen und darstellen    
try:
    if os.path.isfile('mittelwerte_def.pkl'):
        with open('mittelwerte_def.pkl', 'rb') as file:
            [start, ende] = dill.load(file)

        start = np.asarray(start)
        ende = np.asarray(ende)                

        for i in range(np.shape(Zeit)[1]):
            mittelwerte_def[i].value = [start[i], ende[i]]
except:
    pass

# Zeitfenster Gesamt      
mittelwerte_def_ges = widgets.FloatRangeSlider(
    value=[float(config['Import']['gesamt_mittel_min']), float(config['Import']['gesamt_mittel_max'])],
    min=0,
    max=Zeit[-1,0],
    step=0.1,
    description='Gesamt: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
    )        
w_mittelwerte = widgets.Button(
    description='Für alle übernehmen',
    layout = widgets.Layout(width='25%'))

# Berechnung
Filterwert = widgets.IntText(
    value=int(config['Import']['Filterwert']),
    description='Filterwert:',
    disabled=False,
    style = {'description_width': '100px'})
peak_abstand_w = widgets.IntText(
    value=int(config['Import']['Peakabstand']),
    description='Abstand Peaks:',
    disabled=False,
    style = {'description_width': '100px'})

# Ergebnisse
e_ordnerpfad = widgets.Text(
    value= config['Import']['e_ordnerpfad'],
    description='Ergebnisse Pfad:',
    layout = widgets.Layout(width='95%'),
    style = style)
e_dateiname = widgets.Text(
    value= config['Import']['e_dateiname'],
    description='Dateiname:',
    layout = widgets.Layout(width='50%'),
    style = style)

# Tabs konfigurieren
tab1 = VBox(children=[Vorzeichen_Fx_check,
                     Vorzeichen_Fy_check,
                     Vorzeichen_Fz_check,
                     Offset_Fx_check,
                     Offset_Fy_check,
                     Offset_Fz_check])
tab2 = VBox(children=[mittelwerte_def_ges, w_mittelwerte])
tab3 = VBox(mittelwerte_def)
tab4 = VBox(children=[Filterwert, peak_abstand_w])
tab5 = VBox(children=[e_ordnerpfad, e_dateiname])

tab = widgets.Tab(children=[tab1, tab2, tab3, tab4, tab5])
tab.set_title(0, 'Korrekturen')
tab.set_title(1, 'Zeitfenster Gesamt')
tab.set_title(2, 'Zeitfenster Einzeln')
tab.set_title(3, 'Berechnung')
tab.set_title(4, 'Ergebnisse')

# Buttons unter den Tabs
w_berechnen = widgets.Button(
    description='Berechnen',
    layout = widgets.Layout(width='25%'))
speichern = widgets.Button(
    description='Einstellungen Speichern',
    layout = widgets.Layout(width='25%'))
default = widgets.Button(
    description='Default Einstellungen',
    layout = widgets.Layout(width='25%'),)    

output = widgets.Output()

## Funktion der Buttons beim Anklicken ######################################

@w_mittelwerte.on_click
def w_mittelwerte_on_click(b):
    mittelwerte_ges_f()    

@w_berechnen.on_click
def w_berechnen_on_click(b):
    berechnen()
    
@speichern.on_click
def speichern_on_click(b):
    speichern_f()
    print('Erfolgreich gespeichert')

@default.on_click
def default_on_click(b):
    default_f()
    print('Benutzereingabe bitte mit Strg + Enter neu laden')    

# Globale Variablen laden ######################################
try:
    locals().update(auswertung())
except:
    pass  
    
# Darstellung der Tabs muss in der letzten Zeile stehen
VBox(children=[tab, speichern, default, w_berechnen, output])

# Diagramme erstellen
## Allgemeine formatierung

In [111]:
#"Strg + Enter" um die Diagramme standardmäßig zu formatieren
#plt.rcParams.keys() 
import pickle
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import configparser
import dill
import os
from scipy import signal

#%matplotlib inline

try:
    with open('ergebnisse.pkl', 'rb') as file:
        [Zeit, Fx, Fy, Fz, Fx_f, Fy_f, Fz_f, Fx_peaks, Fy_peaks, Fz_peaks, Fx_mean, Fy_mean, Fz_mean, ergebnisse] = dill.load(file)
except:
    print('Wurden schon Ergebnisse berechnet?')

# Diagrammgröße
plt.rcParams["figure.figsize"] = (10/2.54,6/2.54) # /2.54 inch to cm 

# Linien und Marker
plt.rcParams['lines.linestyle'] = '-'
plt.rcParams["lines.linewidth"] = 0
plt.rcParams["lines.markersize"] = .5
plt.rcParams["lines.marker"] = 'o'
plt.rcParams["lines.marker"] = 'o'

#Farben
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])
#plt.rcParams['axes.prop_cycle'] = plt.cycler(color=["r", "#e94cdc", "0.7"])

# Tics
plt.rcParams["axes.grid"] = True
plt.rcParams['axes.axisbelow'] = True
plt.rcParams["xtick.minor.visible"] = True

# Schriftgröße
plt.rcParams['font.size'] = 8
plt.rcParams['axes.titlesize'] = 8
plt.rcParams['axes.labelsize'] = 8
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize']  = 8

# Schriftart
plt.rcParams['legend.fontsize']  = 8

# Legende
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.family"] = "Times New Roman"

# Ausgabeart
#%matplotlib inline  
plt.rcParams['figure.dpi'] = 125 # Regelt auch die Größe des Diagramms in der Notebook Darstellung
plt.rcParams["figure.facecolor"] = "white"

## Kraftsignale über die Zeit

In [110]:
# "Strg + Enter" um die Benutzereingabe anzuzeigen

### Funktionen definieren ######################################        
output1 = widgets.Output()

@output1.capture()
def plotten_f():
    
    # Erst mal Daten importieren
    try:
        with open('ergebnisse.pkl', 'rb') as file:
            [Zeit, Fx, Fy, Fz, Fx_f, Fy_f, Fz_f, Fx_peaks, Fy_peaks, Fz_peaks, Fx_mean, Fy_mean, Fz_mean, ergebnisse] = dill.load(file)
    except:
        print('Wurden schon Ergebnisse berechnet?')

    for Messbahn in range((messbahn_w.value[0]-1),messbahn_w.value[1]):
        
        print('Diagramm zu Messwerten:', (Messbahn+1))

        # Plot über Graphische Oberfläche
        
        plt.rcParams["axes.grid"] = grid_w.value

        fig, ax = plt.subplots() #start des Plots 

        x = Zeit[:,Messbahn]
        
        if y_achse_Fx.value: ax.plot(signal.decimate(x, dezimierung_w.value), 
                                     signal.decimate(Fx[:,Messbahn], dezimierung_w.value), label = 'Fx - Rohdaten')
        if y_achse_Fy.value: ax.plot(signal.decimate(x, dezimierung_w.value), 
                                     signal.decimate(Fy[:,Messbahn], dezimierung_w.value), label = 'Fy - Rohdaten')
        if y_achse_Fz.value: ax.plot(signal.decimate(x, dezimierung_w.value), 
                                     signal.decimate(Fz[:,Messbahn], dezimierung_w.value), label = 'Fz - Rohdaten')
        if y_achse_Fx_f.value: ax.plot(signal.decimate(x, dezimierung_w.value), 
                                       signal.decimate(Fx_f[:,Messbahn], dezimierung_w.value), label = 'Fx - Gefiltert')
        if y_achse_Fy_f.value: ax.plot(signal.decimate(x, dezimierung_w.value), 
                                       signal.decimate(Fy_f[:,Messbahn], dezimierung_w.value), label = 'Fy - Gefiltert')
        if y_achse_Fz_f.value: ax.plot(signal.decimate(x, dezimierung_w.value), 
                                       signal.decimate(Fz_f[:,Messbahn], dezimierung_w.value), label = 'Fz - Gefiltert')
        if y_achse_Fx_mean.value: ax.plot([ergebnisse.iloc[(Messbahn)]['start'], ergebnisse.iloc[(Messbahn)]['ende']], 
                                          [ergebnisse.iloc[(Messbahn)]['Fx_mean'], ergebnisse.iloc[(Messbahn)]['Fx_mean']] ,
                                          label = 'Fx - Mittelwert', lw=2)
        if y_achse_Fy_mean.value: ax.plot([ergebnisse.iloc[(Messbahn)]['start'], ergebnisse.iloc[(Messbahn)]['ende']], 
                                          [ergebnisse.iloc[(Messbahn)]['Fy_mean'], ergebnisse.iloc[(Messbahn)]['Fy_mean']] ,
                                          label = 'Fy - Mittelwert', lw=2)
        if y_achse_Fz_mean.value: ax.plot([ergebnisse.iloc[(Messbahn)]['start'], ergebnisse.iloc[(Messbahn)]['ende']], 
                                          [ergebnisse.iloc[(Messbahn)]['Fz_mean'], ergebnisse.iloc[(Messbahn)]['Fz_mean']] ,
                                          label = 'Fz - Mittelwert', lw=2)
        if y_achse_Fx_peak.value: ax.plot(x, Fx_peaks[:,Messbahn], label = 'Fx - Peaks', lw=0, ms = 1)
        if y_achse_Fy_peak.value: ax.plot(x, Fy_peaks[:,Messbahn], label = 'Fy - Peaks', lw=0, ms = 1)
        if y_achse_Fz_peak.value: ax.plot(x, Fz_peaks[:,Messbahn], label = 'Fz - Peaks', lw=0, ms = 1)
        if y_achse_Fx_peaks_mean.value: ax.plot([ergebnisse.iloc[(Messbahn)]['start'], ergebnisse.iloc[(Messbahn)]['ende']], 
                                          [ergebnisse.iloc[(Messbahn)]['Fx_peaks_mean'], ergebnisse.iloc[(Messbahn)]['Fx_peaks_mean']] ,
                                          label = 'Fx - Mittelwert', lw=2)
        if y_achse_Fy_peaks_mean.value: ax.plot([ergebnisse.iloc[(Messbahn)]['start'], ergebnisse.iloc[(Messbahn)]['ende']], 
                                          [ergebnisse.iloc[(Messbahn)]['Fy_peaks_mean'], ergebnisse.iloc[(Messbahn)]['Fy_peaks_mean']] ,
                                          label = 'Fy - Mittelwert', lw=2)
        if y_achse_Fz_peaks_mean.value: ax.plot([ergebnisse.iloc[(Messbahn)]['start'], ergebnisse.iloc[(Messbahn)]['ende']], 
                                          [ergebnisse.iloc[(Messbahn)]['Fz_peaks_mean'], ergebnisse.iloc[(Messbahn)]['Fz_peaks_mean']] ,
                                          label = 'Fz - Mittelwert', lw=2)
    
        # Achsenlimits
        if y_lim_fest_w.value: ax.set_ylim(y_limits_w.value[0], y_limits_w.value[1])
        if x_lim_fest_w.value: ax.set_xlim(x_limits_w.value[0], x_limits_w.value[1])

        # Beschriftung
        beschriftung = ''

        if probe_w.value: beschriftung = beschriftung + "Probe: "    + str(ergebnisse.iloc[(Messbahn)]['probe']) + "\n"
        if werkezug_w.value: beschriftung = beschriftung + "Werkzeug: " + str(ergebnisse.iloc[(Messbahn)]['werkzeug']) + "\n"
        if korngroesse_w.value: beschriftung = beschriftung + "Korngröße: " + str(ergebnisse.iloc[(Messbahn)]['korngroesse']) + "\n"
        if standweg_w.value: beschriftung = beschriftung + "Standweg: " + str(ergebnisse.iloc[(Messbahn)]['standweg']) + " $m$" + "\n"
        if richtung_w.value: beschriftung = beschriftung + "Richtung: " + str(ergebnisse.iloc[(Messbahn)]['drehrichtung']) + "\n"
        if vc_w.value: beschriftung = beschriftung + "vc: "       + str(ergebnisse.iloc[(Messbahn)]['vc']) + " $m/min$" + "\n"
        if f_w.value: beschriftung = beschriftung + "f: "        + str(ergebnisse.iloc[(Messbahn)]['fu']) + " $µm/U$ " + "\n"
        if ap_w.value: beschriftung = beschriftung + "ap: "       + str(ergebnisse.iloc[(Messbahn)]['ap']) + " $mm$" + "\n"
        if ae_w.value: beschriftung = beschriftung + "ae: "       + str(ergebnisse.iloc[(Messbahn)]['ae']) + " $mm$" + "\n"
        if q_w.value: beschriftung = beschriftung + "Q: "        + str(int(ergebnisse.iloc[(Messbahn)]['q'])) + " $mm^3/min$'"
    
        props = dict(boxstyle='round', facecolor='white', alpha=1, edgecolor ='silver')
        ax.text(1.05, 0.04, beschriftung[:-1], transform=ax.transAxes, fontsize=8,
                verticalalignment='bottom', bbox=props)

        ax.set_xlabel(x_label_w.value)
        ax.set_ylabel(y_label_w.value)

        # Legende aufhübschen
        if legende_w.value:
            legend = ax.legend(loc= legendenpos_w.value) #(framealpha = 0.5, bbox_to_anchor=(1.33,1.02)) 
            for legend_handle in legend.legendHandles:
                legend_handle._legmarker.set_markersize(2)  
            
        plt.show()

        # Diagramm als .png speichern
        filename = speicherort_w.value + '\\' + dateiname_w.value + '_Messung_' + str(Messbahn+1)
        if speichern_w.value: fig.savefig(filename, bbox_inches='tight')
        
        # Wenn der Range slider z.B. auf (1,1) steht
        if messbahn_w.value[0] == messbahn_w.value[1]:
            break
    
## Initialisierung ######################################
# Daten
messbahn_w = widgets.IntRangeSlider(
    value=[1, 1],
    min=1,
    max=len(ergebnisse),
    step=1,
    description='Messwerte:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')
x_lim_fest_w = widgets.Checkbox(
    value=False,
    description='Fixe x-Limits',
    indent=False)
y_lim_fest_w = widgets.Checkbox(
    value=False,
    description='Fixe y-Limits',
    indent=False)
x_limits_w = widgets.FloatRangeSlider(
    min=0,
    max=Zeit[-1,0],
    value=[0, Zeit[-1,0]],
    step=.1,
    description='x-Limits:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f')
y_limits_w = widgets.FloatRangeSlider(
    min=np.min([Fx, Fy, Fz]),
    max=np.max([Fx, Fy, Fz]),
    value=[np.min([Fx, Fy, Fz]), np.max([Fx, Fy, Fz])],
    step=.1,
    description='y-Limits:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f')

dezimierung_w = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='Dezimierung:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')
y_achse_Fx = widgets.Checkbox(
    value=False,
    description='Fx - Rohdaten',
    indent=False)
y_achse_Fy = widgets.Checkbox(
    value=False,
    description='Fy- Rohdaten',
    indent=False)
y_achse_Fz = widgets.Checkbox(
    value=False,
    description='Fz - Rohdaten',
    indent=False)
y_achse_Fx_f = widgets.Checkbox(
    value=True,
    description='Fx - Gefiltert',
    indent=False)
y_achse_Fy_f = widgets.Checkbox(
    value=False,
    description='Fy - Gefiltert',
    indent=False)
y_achse_Fz_f = widgets.Checkbox(
    value=False,
    description='Fz - Gefiltert',
    indent=False)
y_achse_Fx_mean = widgets.Checkbox(
    value=False,
    description='Fx - Mittelwert',
    indent=False)
y_achse_Fy_mean = widgets.Checkbox(
    value=False,
    description='Fy - Mittelwert',
    indent=False)
y_achse_Fz_mean = widgets.Checkbox(
    value=False,
    description='Fz - Mittelwert',
    indent=False)
y_achse_Fx_peak = widgets.Checkbox(
    value=False,
    description='Fx - Peaks',
    indent=False)
y_achse_Fy_peak = widgets.Checkbox(
    value=False,
    description='Fy - Peaks',
    indent=False)
y_achse_Fz_peak = widgets.Checkbox(
    value=False,
    description='Fz - Peaks',
    indent=False)
y_achse_Fx_peaks_mean = widgets.Checkbox(
    value=False,
    description='Fx - Peakmittelwert',
    indent=False)
y_achse_Fy_peaks_mean = widgets.Checkbox(
    value=False,
    description='Fy - Peakmittelwert',
    indent=False)
y_achse_Fz_peaks_mean = widgets.Checkbox(
    value=False,
    description='Fz - Peakmittelwert',
    indent=False)


# Darstellung
probe_w = widgets.Checkbox(
    value=True,
    description='Probe',
    indent=False)
werkezug_w = widgets.Checkbox(
    value=True,
    description='Werkzeug',
    indent=False)
korngroesse_w = widgets.Checkbox(
    value=True,
    description='Korngröße',
    indent=False)
standweg_w = widgets.Checkbox(
    value=True,
    description='Standweg',
    indent=False)
richtung_w = widgets.Checkbox(
    value=True,
    description='Richtung',
    indent=False)
vc_w = widgets.Checkbox(
    value=True,
    description='Schnittgeschwindigkeit vc',
    indent=False)
f_w = widgets.Checkbox(
    value=True,
    description='Vorschub f',
    indent=False)
ap_w = widgets.Checkbox(
    value=True,
    description='Axiale Schnitttiefe ap',
    indent=False)
ae_w = widgets.Checkbox(
    value=True,
    description='Radiale Schnitttiefe ae',
    indent=False)
q_w = widgets.Checkbox(
    value=True,
    description='Zeitspanvolumen Q',
    indent=False)
x_label_w = widgets.Text(
    value= 'Standweg in $m$',
    description='x-Achse',
    layout = widgets.Layout(width='80%'),
    style = {'description_width': '120px'})
y_label_w = widgets.Text(
    value= 'Kraft in $N$',
    description='y-Achse',
    layout = widgets.Layout(width='80%'),
    style = {'description_width': '120px'})
grid_w = widgets.Checkbox(
    value=False,
    description='Gitternetz',
    indent=False)
legende_w = widgets.Checkbox(
    value=True,
    description='Legende',
    indent=False)
legendenpos_w = widgets.Dropdown(
    options=[('oben links', 2), ('oben rechts', 1), ('unten llinks', 3), ('unten rechts', 4), ('automatisch', 0)],
    value=2,
    description='Legendenposition:',
    style = {'description_width': '120px'})

# Speichern
speichern_w = widgets.Checkbox(
    value=True,
    description='Speichern',
    indent=False)
speicherort_w = widgets.Text(
    value= str(os.getcwd()),
    description='Speicherpfad',
    layout = widgets.Layout(width='95%'))
dateiname_w = widgets.Text(
    value= 'Diagramm_1',
    description='Name',
    layout = widgets.Layout(width='95%'))

w_plotten = widgets.Button(
    description='Plotten',
    layout = widgets.Layout(width='25%'))

# Tabs konfigurieren
left_box = VBox([y_achse_Fx, y_achse_Fx_f, y_achse_Fx_mean, y_achse_Fx_peak, y_achse_Fx_peaks_mean])
middle_box = VBox([y_achse_Fy, y_achse_Fy_f, y_achse_Fy_mean, y_achse_Fy_peak, y_achse_Fy_peaks_mean])
right_box = VBox([y_achse_Fz, y_achse_Fz_f, y_achse_Fz_mean, y_achse_Fz_peak, y_achse_Fz_peaks_mean])
daten_box = HBox([left_box, middle_box, right_box])
tab1 = VBox([messbahn_w, dezimierung_w, x_lim_fest_w, x_limits_w, y_lim_fest_w, y_limits_w, daten_box])

box_layout = Layout(width='40%')
left_box = VBox([x_label_w, y_label_w, legende_w, legendenpos_w, grid_w], layout=box_layout)
right_box = VBox([probe_w, werkezug_w, korngroesse_w, standweg_w, richtung_w, vc_w, f_w, ap_w, ae_w, q_w])
tab2 = HBox([left_box, right_box])

tab3 = VBox(children=[speichern_w,
                      speicherort_w,
                      dateiname_w])

tab = widgets.Tab(children=[tab1, tab2, tab3])
tab.set_title(0, 'Daten')
tab.set_title(1, 'Beschriftung')
tab.set_title(2, 'Speichern')

## Funktion der Buttons beim Anklicken ######################################
    
@w_plotten.on_click
def w_plotten_on_click(b):
    # altes Diagramm löschen
    output1.clear_output()  
    plotten_f()

display(output1)

# Darstellung der Tabs muss in der letzten Zeile stehen
VBox(children=[tab, w_plotten])

Output()

## Kraft über Parameter

In [73]:
# "Strg + Enter" um die Benutzereingabe anzuzeigen

### Funktionen definieren ######################################        
output1 = widgets.Output()

@output1.capture()
def plotten_f():
    
    # Erst mal Daten importieren
    try:
        with open('ergebnisse.pkl', 'rb') as file:
            [Zeit, Fx, Fy, Fz, Fx_f, Fy_f, Fz_f, Fx_peaks, Fy_peaks, Fz_peaks, Fx_mean, Fy_mean, Fz_mean,
                       ergebnisse] = dill.load(file)
    except:
        print('Wurden schon Ergebnisse berechnet?')
        
    # Plot über Graphische Oberfläche

    plt.rcParams["axes.grid"] = grid_w.value

    fig, ax = plt.subplots() #start des Plots

    for werkzeug in range((werkzeug_w.value[0]),werkzeug_w.value[1]+1): 
        
        filter_array = []
                
        for i in range(0, len(ergebnisse)):
            if((ergebnisse.iloc[i]['werkzeug'] >= werkzeug_w.value[0]) &
               (ergebnisse.iloc[i]['werkzeug'] <= werkzeug_w.value[1]) &
               (ergebnisse.iloc[i]['ae'] >= ae_w.value[0]) &
               (ergebnisse.iloc[i]['ae'] <= ae_w.value[1]) &
               (ergebnisse.iloc[i]['ap'] >= ap_w.value[0]) &
               (ergebnisse.iloc[i]['ap'] <= ap_w.value[1]) &
               (ergebnisse.iloc[i]['vc'] >= vc_w.value[0]) &
               (ergebnisse.iloc[i]['vc'] <= vc_w.value[1]) &
               (ergebnisse.iloc[i]['fu'] >= fu_w.value[0]) &
               (ergebnisse.iloc[i]['fu'] <= fu_w.value[1]) &
               (ergebnisse.iloc[i]['standweg'] >= standweg_w.value[0]) &
               (ergebnisse.iloc[i]['standweg'] <= standweg_w.value[1])
               ):
                filter_array.append(True)
            else:
                filter_array.append(False)

        if x_werte_w.value == 1: x = ergebnisse.loc[:, 'werkzeug'][filter_array]
        if x_werte_w.value == 2: x = ergebnisse.loc[:, 'ae'][filter_array] 
        if x_werte_w.value == 3: x = ergebnisse.loc[:, 'ap'][filter_array] 
        if x_werte_w.value == 4: x = ergebnisse.loc[:, 'fu'][filter_array] 
        if x_werte_w.value == 5: x = ergebnisse.loc[:, 'vc'][filter_array]
        if x_werte_w.value == 6: x = ergebnisse.loc[:, 'standweg'][filter_array] 

        x_limits_w.min = 0
        x_limits_w.max = 10000
        x_limits_w.min = np.min(x)/2
        x_limits_w.max = np.max(x)*2

        if y_achse_Fx.value: ax.scatter(x, ergebnisse.loc[:, 'Fx_mean'][filter_array], s= 4, 
                                        label = ('Werkzeug: ' + str(werkzeug) + ' - Fx'))
        if y_achse_Fy.value: ax.scatter(x, ergebnisse.loc[:, 'Fy_mean'][filter_array], s= 4,
                                        label = ('Werkzeug: ' + str(werkzeug) + ' - Fy'))
        if y_achse_Fz.value: ax.scatter(x, ergebnisse.loc[:, 'Fz_mean'][filter_array], s= 4,
                                        label = ('Werkzeug: ' + str(werkzeug) + ' - Fz'))
            
    # Achsenlimits
    if y_lim_fest_w.value: ax.set_ylim(y_limits_w.value[0], y_limits_w.value[1])
    if x_lim_fest_w.value: ax.set_xlim(x_limits_w.value[0], x_limits_w.value[1])

    ax.set_xlabel(x_label_w.value)
    ax.set_ylabel(y_label_w.value)

    # Legende aufhübschen
    if legende_w.value: legend = ax.legend(loc= legendenpos_w.value) #(framealpha = 0.5, bbox_to_anchor=(1.33,1.02)) 

    plt.show()

    # Diagramm als .png speichern
    filename = speicherort_w.value + '\\' + dateiname_w.value
    if speichern_w.value: fig.savefig(filename, bbox_inches='tight')
        
def einst_speichern(filter_name):
    print(f'{filter_name} gespeichert')
    
    config = configparser.ConfigParser()
    config.read('filter_config.ini')

    config[filter_name]= {'werkzeug_min': str(werkzeug_w.value[0]),'werkzeug_max': str(werkzeug_w.value[0]),
                         'standweg_min': str(standweg_w.value[0]),'standweg_max': str(standweg_w.value[0]),
                         'ae_min': str(ae_w.value[0]),'ae_max': str(ae_w.value[0]),
                         'ap_min': str(ap_w.value[0]),'ap_max': str(ap_w.value[0]),
                         'fu_min': str(ae_w.value[0]),'fu_max': str(ae_w.value[0]),
                         'vc_min': str(vc_w.value[0]),'vc_max': str(vc_w.value[0]),
                         'x_label': str(x_label_w.value),
                         'speicherort': str(speicherort_w.value),
                         'x_werte': str(x_werte_w.value)}

    with open('filter_config.ini', 'w') as configfile:
        config.write(configfile)
        
def einst_laden(filter_name):
    print(f'{filter_name} geladen')
    config = configparser.ConfigParser()
    config.read('filter_config.ini')
    
    werkzeug_w.value = [float(config[filter_name]['werkzeug_min']), float(config[filter_name]['werkzeug_max'])]
    standweg_w.value = [float(config[filter_name]['standweg_min']), float(config[filter_name]['standweg_max'])]
    ae_w.value = [float(config[filter_name]['ae_min']), float(config[filter_name]['ae_max'])]
    ap_w.value = [float(config[filter_name]['ap_min']), float(config[filter_name]['ap_max'])]
    fu_w.value = [float(config[filter_name]['fu_min']), float(config[filter_name]['fu_max'])]
    vc_w.value = [float(config[filter_name]['vc_min']), float(config[filter_name]['vc_max'])]
    x_label_w.value = config[filter_name]['x_label']
    speicherort_w.value = config[filter_name]['speicherort']
    x_werte_w.value = int(config[filter_name]['x_werte'])
        
## Initialisierung ######################################
# Daten
werkzeug_w = widgets.IntRangeSlider(
    value=[np.min(ergebnisse.iloc[:]['werkzeug']), np.max(ergebnisse.iloc[:]['werkzeug'])],
    min=np.min(ergebnisse.iloc[:]['werkzeug']),
    max=np.max(ergebnisse.iloc[:]['werkzeug']),
    step=1,
    description='Werkzeug:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')
ae_w = widgets.FloatRangeSlider(
    value=[np.min(ergebnisse.iloc[:]['ae']), np.max(ergebnisse.iloc[:]['ae'])],
    min=np.min(ergebnisse.iloc[:]['ae']),
    max=np.max(ergebnisse.iloc[:]['ae']),
    step=.01,
    description='ae:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f')
ap_w = widgets.FloatRangeSlider(
    value=[np.min(ergebnisse.iloc[:]['ap']), np.max(ergebnisse.iloc[:]['ap'])],
    min=np.min(ergebnisse.iloc[:]['ap']),
    max=np.max(ergebnisse.iloc[:]['ap']),
    step=.01,
    description='ap:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f')
fu_w = widgets.FloatRangeSlider(
    value=[np.min(ergebnisse.iloc[:]['fu']), np.max(ergebnisse.iloc[:]['fu'])],
    min=np.min(ergebnisse.iloc[:]['fu']),
    max=np.max(ergebnisse.iloc[:]['fu']),
    step=.1,
    description='fu:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f')
vc_w = widgets.IntRangeSlider(
    value=[np.min(ergebnisse.iloc[:]['vc']), np.max(ergebnisse.iloc[:]['vc'])],
    min=np.min(ergebnisse.iloc[:]['vc']),
    max=np.max(ergebnisse.iloc[:]['vc']),
    step=.01,
    description='vc:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')
standweg_w = widgets.FloatRangeSlider(
    value=[np.min(ergebnisse.iloc[:]['standweg']), np.max(ergebnisse.iloc[:]['standweg'])],
    min=0,
    max=np.max(ergebnisse.iloc[:]['standweg']),
    step=.01,
    description='standweg:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f')

x_werte_w = widgets.Dropdown(
    options=[('Werkzeug', 1), ('ae', 2), ('ap', 3), ('f', 4), ('vc', 5), ('standweg', 6)],
    value=6,
    description='x-Werte:')

x_lim_fest_w = widgets.Checkbox(
    value=False,
    description='Fixe x-Limits',
    indent=False)
y_lim_fest_w = widgets.Checkbox(
    value=False,
    description='Fixe y-Limits',
    indent=False)
x_limits_w = widgets.FloatRangeSlider(
    min=0,
    max=np.max(ergebnisse.loc[:, 'standweg']+0.1),
    value=[0, np.max(ergebnisse.loc[:, 'standweg'])+0.1],
    step=.1,
    description='x-Limits:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f')
y_limits_w = widgets.FloatRangeSlider(
    min=np.min([Fx_mean, Fy_mean, Fz_mean]),
    max=(np.max([Fx_mean, Fy_mean, Fz_mean])*1.1),
    value=[np.min([Fx_mean, Fy_mean, Fz_mean]), np.max([Fx_mean, Fy_mean, Fz_mean])],
    step=.1,
    description='y-Limits:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f')

dezimierung_w = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='Dezimierung:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')
y_achse_Fx = widgets.Checkbox(
    value=False,
    description='Fx',
    indent=False)
y_achse_Fy = widgets.Checkbox(
    value=True,
    description='Fy',
    indent=False)
y_achse_Fz = widgets.Checkbox(
    value=False,
    description='Fz',
    indent=False)

# Darstellung
x_label_w = widgets.Text(
    value= 'Standweg in $m$',
    description='x-Achse',
    layout = widgets.Layout(width='80%'),
    style = {'description_width': '120px'})
y_label_w = widgets.Text(
    value= 'Kraft in $N$',
    description='y-Achse',
    layout = widgets.Layout(width='80%'),
    style = {'description_width': '120px'})
grid_w = widgets.Checkbox(
    value=False,
    description='Gitternetz',
    indent=False)
legende_w = widgets.Checkbox(
    value=True,
    description='Legende',
    indent=False)
legendenpos_w = widgets.Dropdown(
    options=[('oben links', 2), ('oben rechts', 1), ('unten llinks', 3), ('unten rechts', 4), ('automatisch', 0)],
    value=2,
    description='Legendenposition:',
    style = {'description_width': '120px'})

# Speichern
speichern_w = widgets.Checkbox(
    value=True,
    description='Speichern',
    indent=False)
speicherort_w = widgets.Text(
    value= str(os.getcwd()),
    description='Speicherort',
    layout = widgets.Layout(width='95%'))
dateiname_w = widgets.Text(
    value= 'Diagramm_2',
    description='Dateiname',
    layout = widgets.Layout(width='50%'))
standweg_s = widgets.Button(
    description='Standweg Speichern',
    layout = widgets.Layout(width='95%'))
standweg_l = widgets.Button(
    description='Standweg Laden',
    layout = widgets.Layout(width='95%'))
ae_s = widgets.Button(
    description='ae Speichern',
    layout = widgets.Layout(width='95%'))
ae_l = widgets.Button(
    description='ae Laden',
    layout = widgets.Layout(width='95%'))
ap_s = widgets.Button(
    description='ap Speichern',
    layout = widgets.Layout(width='95%'))
ap_l = widgets.Button(
    description='ap Laden',
    layout = widgets.Layout(width='95%'))
fu_s = widgets.Button(
    description='fu Speichern',
    layout = widgets.Layout(width='95%'))
fu_l = widgets.Button(
    description='fu Laden',
    layout = widgets.Layout(width='95%'))
vc_s = widgets.Button(
    description='vc Speichern',
    layout = widgets.Layout(width='95%'))
vc_l = widgets.Button(
    description='vc Laden',
    layout = widgets.Layout(width='95%'))

# Plotten
w_plotten = widgets.Button(
    description='Plotten',
    layout = widgets.Layout(width='25%'))

# Tabs konfigurieren
left_box = VBox([y_achse_Fx])
middle_box = VBox([y_achse_Fy])
right_box = VBox([y_achse_Fz])
daten_box = HBox([left_box, middle_box, right_box])
tab1 = VBox([werkzeug_w, standweg_w, ae_w, ap_w, fu_w, vc_w, 
             x_werte_w, x_lim_fest_w, x_limits_w, y_lim_fest_w, y_limits_w, daten_box])

box_layout = Layout(width='50%')
left_box = VBox([x_label_w, y_label_w, legende_w, legendenpos_w, grid_w], layout=box_layout)
right_box = VBox([])
tab2 = HBox([left_box, right_box])

box_layout = Layout(width='25%')
left_box = VBox([standweg_s, ae_s, ap_s, fu_s, vc_s], layout=box_layout)
right_box = VBox([standweg_l, ae_l, ap_l, fu_l, vc_l], layout=box_layout)
daten_box = HBox([left_box, right_box])
tab3 = VBox(children=[speichern_w,
                      speicherort_w,
                      dateiname_w,
                     daten_box])

tab = widgets.Tab(children=[tab1, tab2, tab3])
tab.set_title(0, 'Daten')
tab.set_title(1, 'Beschriftung')
tab.set_title(2, 'Speichern')

## Funktion der Buttons beim Anklicken ######################################
    
@w_plotten.on_click
def w_plotten_on_click(b):
    # altes Diagramm löschen
    output1.clear_output()  
    plotten_f()

@standweg_s.on_click
def standweg_s_on_click(b):
    einst_speichern('standweg')
@standweg_l.on_click
def standweg_l_on_click(b):
    einst_laden('standweg')
@ae_s.on_click
def ae_s_on_click(b):
    einst_speichern('ae')
@ae_l.on_click
def ae_l_on_click(b):
    einst_laden('ae')
@ap_s.on_click
def ap_s_on_click(b):
    einst_speichern('ap')
@ap_l.on_click
def ap_l_on_click(b):
    einst_laden('ap')
@fu_s.on_click
def fu_s_on_click(b):
    einst_speichern('fu')
@fu_l.on_click
def fu_l_on_click(b):
    einst_laden('fu')
@vc_s.on_click
def vc_s_on_click(b):
    einst_speichern('vc')
@vc_l.on_click
def vc_l_on_click(b):
    einst_laden('vc')

display(output1)

# Darstellung der Tabs muss in der letzten Zeile stehen
VBox(children=[tab, w_plotten])

Output()

### Trendline

In [ ]:
# Trend
# Bei Gleich/Gegenlauf kann kein Trend erzeugt werden
if True:
    try:
        [m,t] = np.polyfit(np.asarray(x),np.asarray(y), 1)
        x = [min(x), max(x)]
        y = [(m*x[0]+t), (m*x[1]+t)]
        pos = ax.plot(x, y, label='trend', lw = 1, ms = 0, ls = '--')
        legend = ax.legend(loc= 'upper left')
        del m, t
    except:
        print("Trendline konnte nicht erzeugt werden")
        ax.set_xlim(-1, 2)

## Oberflächen Ergebnisse

In [ ]:
plt.rcParams["lines.markersize"] = 2.5
[a,b] = Zeit.shape

fig, ax = plt.subplots() #start des Plots

# Werkzeug 1
x = ergebnisse.Fy_mean[ergebnisse['Ra-L'].notna()][ergebnisse.werkzeug == 1]
y = ergebnisse['Ra-L'][ergebnisse['Ra-L'].notna()][ergebnisse.werkzeug == 1]
pos = ax.scatter(x, y, label = 'Werkzeug 1')
y = ergebnisse['Ra-R'][ergebnisse['Ra-L'].notna()][ergebnisse.werkzeug == 1]
pos = ax.scatter(x, y, c='C0')
#pos = ax.scatter(x[y>10], y[y>10], label = 'Ausreißer Werkzeug 2')

ax.axhline(y=0, color='k', lw=0.75, ms=0)

# Achsenlimits
#ax.set_ylim(-5, 25)
#ax.set_xlim(0, 2.5)

# Beschriftung
#ax.set_title('1. Werkzeug')
ax.set_xlabel('Kraft Fy in $N$')
ax.set_ylabel('Kantenbereich in $µm$')

# Legende aufhübschen
legend = ax.legend(loc= 'upper left') #(framealpha = 0.5, bbox_to_anchor=(1.33,1.02)) 
#for legend_handle in legend.legendHandles:
#    legend_handle._legmarker.set_markersize(2)

plt.show()

# Diagramm als .png speichern
fig.savefig('Kante_ueber_Kraft', bbox_inches='tight')

#del fig, ax, pos